# Imports

In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import os
import import_ipynb
import Dataprep

# Scraping Game Data From NBA Reference
Scraping game data from the 2015-16 season through 2018-19 from https://www.basketball-reference.com/. Then web pages are saved in the current working directory/NBARefFiles

In [ ]:
years_grabbed = ['2016', '2017', '2018','2019']

for year in years_grabbed:
    for team in Dataprep.team_handles_dict.values():
        if team != 'CHA':
            gid_list = Dataprep.get_game_id_for_team(team, year)
            for gid in gid_list:
                pagelink = "https://www.basketball-reference.com/boxscores/"+gid+".html"
                page = requests.get(pagelink)
                file = open(os.getcwd()+("/NBARefFiles/"+gid+'.html'), "w")
                file.write(page.text)

# Create a CSV from NBA Reference Game Files
This CSV is later used to make sure we only scrape projections, betting data and drafkings data for games that we have

In [ ]:
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd
import import_ipynb
import Dataprep

home_away_dict = {0: 'away', 1: 'home'}

df = pd.DataFrame()

for file in os.listdir(os.getcwd()+("/NBARefFiles/")):
    try:
        box = Dataprep.get_box_score_stats(file)
        df = pd.concat([df, (box)],ignore_index=True)
    except:
        pass
df.set_index('GID',inplace=True)
df.sort_index(ascending=True,inplace=True)
df.to_csv('outputs/NBABoxStats.csv')   


# Scraping Vegas Odds, Fantasy Projections, and Draftkings scores 
Scraping http://www.dailyfantasyfuel.com for historical fantasy point projections for each player, http://rotoguru1.com for Draftkings salary information, and https://www.sportsbookreview.com for vegas game odds

In [ ]:
df = pd.read_csv("outputs/NBABoxStats.csv")
dates = df['Date'].to_numpy()
date_list = Dataprep.get_dates_list(dates)
Dataprep.saveOddsFiles(dates)
Dataprep.savePlayerProjections(date_list)
Dataprep.saveDKFiles(date_list)